In [34]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lex-fridman-podcast-transcript/podcastdata_dataset.csv


In [35]:
import pandas as pd
archivo_csv = '/kaggle/input/lex-fridman-podcast-transcript/podcastdata_dataset.csv'
df = pd.read_csv(archivo_csv)
df.head()

,id,guest,title,text
0,1,Max Tegmark,Life 3.0,"As part of MIT course 6S099, Artificial Genera..."
1,2,Christof Koch,Consciousness,As part of MIT course 6S099 on artificial gene...
2,3,Steven Pinker,AI in the Age of Reason,"You've studied the human mind, cognition, lang..."
3,4,Yoshua Bengio,Deep Learning,What difference between biological neural netw...
4,5,Vladimir Vapnik,Statistical Learning,The following is a conversation with Vladimir ...


In [36]:
tokens = []
for i in range(len(df)):
    tokens.append(df['text'][i].split())

In [37]:
#Oraciones
len(df['text'][0].split('.'))

611

In [38]:
#Palabras
for i in range(len(df)):
    len(df['text'][i].split())

In [39]:
texto = str(df[df['id'] == 2]['text'].values[0])
len(texto.split())

10217

In [40]:
def wc(texto):
    return len(texto.split())

In [41]:
df['wc'] = df['text'].apply(wc)

In [42]:
def ws(texto):
    return texto.split('.'),len(texto.split('.'))

In [43]:
df[['text sentence', 'ws']] = df['text'].apply(lambda x: pd.Series(ws(x)))


In [44]:
df.head()

,id,guest,title,text,wc,text sentence,ws
0,1,Max Tegmark,Life 3.0,"As part of MIT course 6S099, Artificial Genera...",13424,"[As part of MIT course 6S099, Artificial Gener...",611
1,2,Christof Koch,Consciousness,As part of MIT course 6S099 on artificial gene...,10217,[As part of MIT course 6S099 on artificial gen...,499
2,3,Steven Pinker,AI in the Age of Reason,"You've studied the human mind, cognition, lang...",5989,"[You've studied the human mind, cognition, lan...",292
3,4,Yoshua Bengio,Deep Learning,What difference between biological neural netw...,5993,[What difference between biological neural net...,311
4,5,Vladimir Vapnik,Statistical Learning,The following is a conversation with Vladimir ...,6374,[The following is a conversation with Vladimir...,514


In [45]:
df['ws'].sum()

390883

In [46]:
rows = []
for _, row in df.iterrows():
    episode_id = row['id']
    for idx, sentence in enumerate(row['text sentence'], start=1):
        rows.append({'ep_id': episode_id, 'sentence_id': idx, 'text': sentence})

df_modelado = pd.DataFrame(rows)

In [47]:
oraciones_episodio_1 = df_modelado[df_modelado['ep_id'] == 1]
print(oraciones_episodio_1)

     ep_id  sentence_id                                               text
0        1            1  As part of MIT course 6S099, Artificial Genera...
1        1            2                      He is a professor here at MIT
2        1            3   He's a physicist, spent a large part of his c...
3        1            4   But he's also studied and delved into the ben...
4        1            5   Amongst many other things, he is the cofounde...
..     ...          ...                                                ...
606      1          607   So speaking of software that nobody understan...
607      1          608   But what are your thoughts on deep learning? ...
608      1          609   What are your thoughts about the promise limi...
609      1          610   One of them is when you look at the human bra...
610      1          611                                                   

[611 rows x 3 columns]


In [48]:
# 1. Tokenización: Convertir frases en listas de palabras
df_modelado['tokens'] = df_modelado['text'].apply(lambda x: word_tokenize(x.lower()))

# 2. Entrenamiento de Word2Vec
# Crear una lista de listas de palabras (tokens) para entrenar Word2Vec
sentences = df_modelado['tokens'].tolist()

# Entrenar el modelo Word2Vec
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)



In [49]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [50]:
import pandas as pd
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk


# 3. Obtener embeddings para cada frase
# Función para promediar los vectores de palabras en cada oración
def obtener_embedding(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if vectors:
        return sum(vectors) / len(vectors)  # Promedio de vectores
    else:
        return [0] * model.vector_size

In [51]:
df_modelado['embedding'] = df_modelado['tokens'].apply(lambda x: obtener_embedding(x, model))

In [52]:
df_modelado

,ep_id,sentence_id,text,tokens,embedding
0,1,1,"As part of MIT course 6S099, Artificial Genera...","[as, part, of, mit, course, 6s099, ,, artifici...","[0.19861163, 0.22445412, 0.59233624, -0.243688..."
1,1,2,He is a professor here at MIT,"[he, is, a, professor, here, at, mit]","[0.35261315, 0.65616363, 0.33231214, -0.410016..."
2,1,3,"He's a physicist, spent a large part of his c...","[he, 's, a, physicist, ,, spent, a, large, par...","[-0.4165765, 0.32829908, 0.48733902, 0.1845713..."
3,1,4,But he's also studied and delved into the ben...,"[but, he, 's, also, studied, and, delved, into...","[-0.37365314, -0.5530812, 0.66169035, -0.64571..."
4,1,5,"Amongst many other things, he is the cofounde...","[amongst, many, other, things, ,, he, is, the,...","[-0.30308935, -0.17375223, 0.26148164, -0.1791..."
...,...,...,...,...,...
390878,325,1727,It's the beginning,"[it, 's, the, beginning]","[0.063448906, 0.6749759, 0.81553847, -0.434464..."
390879,325,1728,"It's not the whole story by any means, but it...","[it, 's, not, the, whole, story, by, any, mean...","[-0.17980759, -0.34563464, 0.7182552, -0.23193..."
390880,325,1729,Where's state stored of the system? Is it in ...,"[where, 's, state, stored, of, the, system, ?,...","[-0.12897475, 0.11541487, 0.8889587, -0.710236..."
390881,325,1730,So there are chemical networks,"[so, there, are, chemical, networks]","[0.3387075, -1.0429316, 0.19270645, -0.0287592..."


In [53]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

embeddings = np.array(df_modelado['embedding'].tolist())

# Número de clusters 
n_clusters = 5

# Aplicar K-Means
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
df_modelado['cluster'] = kmeans.fit_predict(embeddings)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [54]:
df_modelado[df_modelado['ep_id'] == 1]

,ep_id,sentence_id,text,tokens,embedding,cluster
0,1,1,"As part of MIT course 6S099, Artificial Genera...","[as, part, of, mit, course, 6s099, ,, artifici...","[0.19861163, 0.22445412, 0.59233624, -0.243688...",0
1,1,2,He is a professor here at MIT,"[he, is, a, professor, here, at, mit]","[0.35261315, 0.65616363, 0.33231214, -0.410016...",0
2,1,3,"He's a physicist, spent a large part of his c...","[he, 's, a, physicist, ,, spent, a, large, par...","[-0.4165765, 0.32829908, 0.48733902, 0.1845713...",0
3,1,4,But he's also studied and delved into the ben...,"[but, he, 's, also, studied, and, delved, into...","[-0.37365314, -0.5530812, 0.66169035, -0.64571...",0
4,1,5,"Amongst many other things, he is the cofounde...","[amongst, many, other, things, ,, he, is, the,...","[-0.30308935, -0.17375223, 0.26148164, -0.1791...",0
...,...,...,...,...,...,...
606,1,607,So speaking of software that nobody understan...,"[so, speaking, of, software, that, nobody, und...","[-0.15644458, -0.12002514, 0.52115595, -0.0703...",0
607,1,608,But what are your thoughts on deep learning? ...,"[but, what, are, your, thoughts, on, deep, lea...","[-0.09975407, -0.10085674, 0.75640166, -0.0454...",3
608,1,609,What are your thoughts about the promise limi...,"[what, are, your, thoughts, about, the, promis...","[0.25803897, -0.4572116, 0.67869514, -0.127189...",0
609,1,610,One of them is when you look at the human bra...,"[one, of, them, is, when, you, look, at, the, ...","[-0.06427292, -0.3188045, 0.48247167, -0.53355...",0


In [55]:
from sklearn.metrics.pairwise import cosine_similarity

# Obtener los centroides de cada cluster
centroides = kmeans.cluster_centers_

# Función para encontrar la similitud coseno entre un vector y el centroide de un cluster
def obtener_similitud_con_centroide(embedding, centroide):
    return cosine_similarity([embedding], [centroide])[0][0]

# Crear un diccionario para almacenar las oraciones más representativas de cada cluster
oraciones_topico = {}

# Iterar sobre cada cluster y encontrar las oraciones más cercanas al centroide
for cluster_id in range(n_clusters):
    # Obtener las oraciones en este cluster
    oraciones_cluster = df_modelado[df_modelado['cluster'] == cluster_id]
    
    # Obtener los embeddings de estas oraciones
    embeddings_cluster = np.array(oraciones_cluster['embedding'].tolist())
    
    # Calcular la similitud coseno entre cada embedding y el centroide del cluster
    similitudes = [obtener_similitud_con_centroide(embedding, centroides[cluster_id]) for embedding in embeddings_cluster]
    
    # Encontrar el índice de la oración más cercana al centroide
    indice_max_similitud = np.argmax(similitudes)
    
    # Obtener la oración más representativa y asociarla con el cluster
    oracion_representativa = oraciones_cluster.iloc[indice_max_similitud]['text']
    
    # Guardar la oración más representativa en el diccionario
    oraciones_topico[cluster_id] = oracion_representativa

# Mostrar las oraciones más representativas de cada cluster (tópico)
for cluster_id, oracion in oraciones_topico.items():
    print(f"Cluster {cluster_id}: {oracion}")


Cluster 0:  If you end up with a list of like, well, these are things we never want to see, that list leaks, and after the passage of some time, certainly couldn't be done today, but after the passage of some time, lots and lots of people in academic labs going all the way down to the high school level are in a position to make it overly simplistic, hit print on a genome and have the virus bearing that genome pop out on the other end and you've got something to worry about, but in general, computational biology I think is incredibly important, particularly because the crushing majority of work that people are doing with the protein folding problem and other things are about creating therapeutics, about creating things that will help us live better, live longer, thrive, be more well, and so forth, and the protein folding problem is a monstrous computational challenge that we seem to make just the most glacial project on, I'm sorry, progress on for years and years, but I think there's a 

In [68]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Crear un DataFrame con los tópicos y sus oraciones representativas
topicos_df = pd.DataFrame(list(oraciones_topico.items()), columns=['Cluster', 'Oracion Representativa'])

def obtener_embedding_busqueda(clave, modelo):
    tokens = clave.split()
    vectores = [modelo.wv[word] for word in tokens if word in modelo.wv]
    if len(vectores) > 0:
        return np.mean(vectores, axis=0)
    else:
        return np.zeros(modelo.vector_size)

# Función para realizar la búsqueda
def buscar_por_topico(clave, modelo, df_modelado, top_n=5):
    # Obtener el embedding de las palabras clave
    embedding_busqueda = obtener_embedding_busqueda(clave, modelo)
    
    # Calcular la similitud coseno entre el embedding de búsqueda y los embeddings de las oraciones
    embeddings_oraciones = np.array(df_modelado['embedding'].tolist())
    similitudes = cosine_similarity([embedding_busqueda], embeddings_oraciones)[0]
    
    # Ordenar las oraciones por similitud
    df_modelado['similitud'] = similitudes
    resultados = df_modelado[['ep_id', 'sentence_id', 'text', 'similitud']].sort_values(by='similitud', ascending=False).head(top_n)
    
    return resultados

# Realizar la búsqueda
clave_busqueda = "deep learning"
resultados_busqueda = buscar_por_topico(clave_busqueda, model, df_modelado)